# Quantum Process Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** November 8, 2018 

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.tools.qi.qi import state_fidelity, outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('X',), ('Zp',)): {'0': 2000},
 (('Y',), ('Zp',)): {'0': 1004, '1': 996},
 (('Z',), ('Zp',)): {'0': 999, '1': 1001},
 (('X',), ('Zm',)): {'1': 2000},
 (('Y',), ('Zm',)): {'0': 988, '1': 1012},
 (('Z',), ('Zm',)): {'0': 1021, '1': 979},
 (('X',), ('Xp',)): {'0': 1008, '1': 992},
 (('Y',), ('Xp',)): {'0': 1023, '1': 977},
 (('Z',), ('Xp',)): {'0': 2000},
 (('X',), ('Yp',)): {'0': 1013, '1': 987},
 (('Y',), ('Yp',)): {'1': 2000},
 (('Z',), ('Yp',)): {'0': 1012, '1': 988}}

In [3]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.fitters.cvx_fit(data, basis, PSD=True, trace_preserving=True, trace=2)
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.16708779335021973
fit fidelity: 0.9940903425322384

CVXOPT Fitter
fit time: 0.19166779518127441
fit fidelity: 0.999897980786586


## 2-Qubit entangling circuit

In [5]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.04325985908508301
fit fidelity: 0.986192298563409

CVXOPT Fitter
fit time: 1.4843859672546387
fit fidelity: 0.9999431082785937


## Using SIC-POVM preparation basis

In [6]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('X',), ('S0',)): {'0': 2000},
 (('Y',), ('S0',)): {'0': 1051, '1': 949},
 (('Z',), ('S0',)): {'0': 980, '1': 1020},
 (('X',), ('S1',)): {'0': 640, '1': 1360},
 (('Y',), ('S1',)): {'0': 1015, '1': 985},
 (('Z',), ('S1',)): {'0': 1943, '1': 57},
 (('X',), ('S2',)): {'0': 680, '1': 1320},
 (('Y',), ('S2',)): {'0': 1838, '1': 162},
 (('Z',), ('S2',)): {'0': 508, '1': 1492},
 (('X',), ('S3',)): {'0': 638, '1': 1362},
 (('Y',), ('S3',)): {'0': 171, '1': 1829},
 (('Z',), ('S3',)): {'0': 550, '1': 1450}}

In [7]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.002238035202026367
fit fidelity: 0.9947095142519727

CVXOPT Fitter
fit time: 0.040667057037353516
fit fidelity: 0.9999304277236141
